In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.drop('customerID',inplace = True , axis = 1)
df.dtypes

In [ ]:
df['TotalCharges'].values

In [ ]:
df['MonthlyCharges'].values

In [ ]:
df[pd.to_numeric(df['TotalCharges'],errors = 'coerce').isnull()]

In [ ]:
df[pd.to_numeric(df['TotalCharges'],errors = 'coerce').isnull()].shape

In [ ]:
df1 = df[df['TotalCharges']!=' ']
df1.shape

In [ ]:
df1.dtypes

In [ ]:
df1['TotalCharges'] = pd.to_numeric(df1['TotalCharges'])

In [ ]:
df1[df1.Churn=='No']

In [ ]:
tenure_churn_no = df1[df.Churn == 'No'].tenure
tenure_churn_yes = df1[df.Churn == 'Yes'].tenure

In [ ]:
plt.hist([tenure_churn_yes , tenure_churn_no] , color = ['green','red'],label = ['Churn = yes' ,'Churn = No'])

plt.xlabel('Tenure')
plt.ylabel('Number of Customers')
plt.legend()
plt.show()

In [ ]:
monthly_charge_churn_no = df1[df.Churn == 'No'].MonthlyCharges
monthly_charge_churn_yes = df1[df.Churn == 'Yes'].MonthlyCharges

In [ ]:
plt.hist([monthly_charge_churn_yes , monthly_charge_churn_no] , color = ['red','green'],label = ['Churn = yes' ,'Churn = No'])

plt.xlabel('Monthly Charges')
plt.ylabel('Number of Customers')
plt.legend()
plt.show()

In [ ]:
def print_unique_col_values(df1):
    for column in df : 
        if df[column].dtypes == 'object':
            print(f'{column} : {df[column].unique()}')
print_unique_col_values(df1)

In [ ]:
df1.replace('No internet service','No',inplace = True)
df1.replace('No phone service','No',inplace = True)

In [ ]:
for column in df1 : 
    print(df1[column].unique())

In [ ]:
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup','DeviceProtection',
                  'TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']

In [ ]:
for col in yes_no_columns:
    df1[col].replace({'Yes':1,'No':0},inplace = True)

In [ ]:
for column in df1 : 
    if df1[column].dtypes == 'object':
        print(f"{column}:{df1[column].unique()}")

In [ ]:
df1['gender'].replace({'Female':1 , 'Male':0},inplace = True)

In [ ]:
df1['gender'].unique()

In [ ]:
s = (df1.dtypes == 'object')
object_cols = list(s[s].index)

print('Categorical Variables: ', object_cols)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

df2 = df1.copy()

encoder = OrdinalEncoder()

df2[object_cols] = encoder.fit_transform(df2[object_cols])

df2.columns = df1.columns 


In [ ]:
df2.dtypes

In [ ]:
df2.head()

In [ ]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']


from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

df2[cols_to_scale] = minmax.fit_transform(df2[cols_to_scale])

In [ ]:
features = df2.drop('Churn',axis = 1)
label = df2['Churn']

In [ ]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test = train_test_split(features,label,test_size = 0.2,random_state = 42)
print('x train shape :',x_train.shape)
print('y train shape :',y_train.shape)
print('x test shape :',x_test.shape)
print('y test shape :',y_test.shape)

In [ ]:
import tensorflow as tf 
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(20,input_shape = (19,),activation = 'relu'),
    keras.layers.Dense(1,activation = 'sigmoid')

])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model.fit(x_train,y_train,epochs = 100)

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
yp = model.predict(x_test)

In [ ]:
y_pred = [ ]

for element in yp:
    if element > 0.5 : 
        y_pred.append(1)
    else : 
        y_pred.append(0)

In [ ]:
y_pred[0:10]

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,y_pred))

In [ ]:
score = accuracy_score(y_test,y_pred)
print('accuracy score : ',score)